In [1]:
import pandas as pd
import numpy as np
import os
from encoding_function import low_frequency_to_others
from sklearn.model_selection import KFold
from category_encoders import TargetEncoder

import warnings
warnings.filterwarnings('ignore')

### 데이터 불러오기
- 학습: 2023년도 이전
- 검증: 2023년
- 테스트: 2024년

In [2]:
def add_y(df):
    df['rank'] = df['rank'].replace(0, 6)

    target_cols = [
        '단승', '복승', '삼복승'
    ]
    for i, col in enumerate(target_cols):
        condition_target = df['rank'] <= i+1
        df.loc[condition_target, col] = 1

    df[target_cols] = df[target_cols].fillna(0)
    df.drop(['rank'], axis=1, inplace=True)
    return df

def add_y_(df):
    df['rank'] = df['rank'].replace(0, 6)

    target_cols = [
        '단승', '복승', '삼복승'
    ]
    for i, col in enumerate(target_cols):
        condition_target = df['rank'] <= i+1
        df.loc[condition_target, col] = df['rank']

    df[target_cols] = df[target_cols].fillna(0)
    df.drop(['rank'], axis=1, inplace=True)
    return df



ROOT_DIR = "data"
RANDOM_STATE = 42

train = pd.read_csv(os.path.join(ROOT_DIR, "train.csv"))
test = pd.read_csv(os.path.join(ROOT_DIR, "test.csv"))

train['성별'] = train['성별'].map({'남': 0, '여': 1})
test['성별'] = test['성별'].map({'남': 0, '여': 1})

train = add_y(train)
test = add_y(test)

print(train.shape, test.shape)
train.head()

(58344, 46) (6528, 46)


,Race_ID,연도,회차,일차,경주번호,경기종류,번호,등급,기수,선수명,...,모터_연대율3,전탑승선수1,전탑승선수2,보트번호,보트_평균착순점,보트_연대율,특이사항,단승,복승,삼복승
0,2016_1_1_1,2016,1,1,1,플라잉,1,A1,1,정용진,...,0.0,NaN,NaN,27,0.0,0.0,NaN,0.0,0.0,1.0
1,2016_1_1_1,2016,1,1,1,플라잉,2,A1,13,김민준,...,0.0,NaN,NaN,50,0.0,0.0,NaN,1.0,1.0,1.0
2,2016_1_1_1,2016,1,1,1,플라잉,3,B2,11,김현덕,...,0.0,NaN,NaN,71,0.0,0.0,주선보류 후 출전,0.0,0.0,0.0
3,2016_1_1_1,2016,1,1,1,플라잉,4,A1,8,한종석,...,0.0,NaN,NaN,53,0.0,0.0,NaN,0.0,1.0,1.0
4,2016_1_1_1,2016,1,1,1,플라잉,5,B1,9,정재용,...,0.0,NaN,NaN,64,0.0,0.0,NaN,0.0,0.0,0.0


In [3]:
def add_weight_penalty(df):
    weight_limit_male = 55
    weight_limit_female = 51
    df['체중'] = df['체중'].astype(float)

    # 중량 부과 여부 계산 (성별에 따라 다른 기준 적용)
    df['중량부과여부'] = df.apply(lambda row: 1 if (row['성별'] == 0 and row['체중'] < weight_limit_male) or
                                             (row['성별'] == 1 and row['체중'] < weight_limit_female) else 0, axis=1)

    # 부과된 중량 계산 (성별에 따라 차감된 체중을 계산)
    df['부과된중량'] = df.apply(lambda row: max(0, weight_limit_male - row['체중']) if row['성별'] == 0 else
    max(0, weight_limit_female - row['체중']), axis=1)

    df.drop('체중', axis=1, inplace=True)

    return df

train = add_weight_penalty(train)
test = add_weight_penalty(test)

In [4]:
def bin_age(df):
    # 나이를 5개 구간으로 나누어 bin화 (예: 0-20, 20-30, 30-40, 40-50, 50+)
    bins = [20, 30, 40, 50, 100]
    labels = ['20-30', '30-40', '40-50', '50+']

    # 나이 변수를 bin화
    df['나이'] = pd.cut(df['나이'], bins=bins, labels=labels, right=False)

    map_age = {
        '20-30':25,
        '30-40':35,
        '40-50':45,
        '50+':55,
    }
    df['나이'] = df['나이'].map(map_age)

    return df

train = bin_age(train)
test = bin_age(test)

## 사용하지 않는 feature drop

- 전일 성적을 어떻게 반영할지

In [5]:
def drop_columns_from_datasets(df):
    drop_cols = [
        '연도', '회차', '일차', '경주번호', # 일차도 제거?
        '금일출주경주',
        '모터번호', '전탑승선수1', '전탑승선수2',
        '보트번호', '특이사항',
        'FL' # F뒤의 숫자는 현재 반기의 실격 횟수, L뒤의 숫자는 반칙횟수?
    ]

    df = df.drop(drop_cols, axis=1)

    return df


train = drop_columns_from_datasets(train)
test = drop_columns_from_datasets(test)

print(train.shape, test.shape)

(58344, 36) (6528, 36)


In [6]:
def reverse_rank_values(df_train, df_val):
    cols_to_reverse = [
        '최근6회차_평균착순점', '최근6회차_평균득점',
        '연간성적_평균착순점',
        '모터_평균착순점',
        '보트_평균착순점'
    ]
    for col in cols_to_reverse:
        max_rank = df_train[col].max()
        df_train[col] = max_rank - df_train[col]
        df_val[col] = max_rank - df_val[col]

        df_train[col].fillna(0, inplace=True)
        df_val[col].fillna(0, inplace=True)

    return df_train, df_val


train, test = reverse_rank_values(train, test)

In [7]:
import re

def extract_numbers(result):
    if isinstance(result, str):  # result가 문자열인 경우에만 처리
        return re.findall(r'-(\d+)-', result)  # -숫자- 형식에서 중간 숫자 추출
    return []  # 문자열이 아니면 빈 리스트 반환

def calculate_mean(numbers):
    numbers = [int(num) for num in numbers]  # 리스트 안의 문자열 숫자를 정수로 변환
    if len(numbers) > 0:
        return np.mean(numbers)  # 리스트가 비어있지 않으면 평균 계산
    else:
        return np.nan  # 빈 리스트인 경우 NaN 반환

def last_race_process(df_train, df_val):
    df_train['전일성적'] = df_train['전일성적'].apply(extract_numbers)  # 순위만 추출 (코스도 같이 추출?)
    df_train['전일성적'] = df_train['전일성적'].apply(calculate_mean) # 평균 계산

    df_val['전일성적'] = df_val['전일성적'].apply(extract_numbers)
    df_val['전일성적'] = df_val['전일성적'].apply(calculate_mean)

    max_rank = df_train['전일성적'].max()
    df_train['전일성적'] = max_rank - df_train['전일성적'] + 1
    df_val['전일성적'] = max_rank - df_val['전일성적'] + 1

    df_train['전일성적'].fillna(1, inplace=True)
    df_val['전일성적'].fillna(1, inplace=True)

    return df_train, df_val

train, test = last_race_process(train, test)

## 일부 숫자형 변수 변환

- 코스별 성적/경기수 분리

In [8]:
def separation_course(df):
    col_list = [
        '코스_1코스', '코스_2코스', '코스_3코스', '코스_4코스', '코스_5코스', '코스_6코스'
    ]
    for col in col_list:
        df[[f'{col[3:]}_성적', f'{col[3:]}_경기수']] = df[col].fillna('').str.split('/', expand=True)

    df.drop(col_list, axis=1, inplace=True)

    return df



train = separation_course(train)
test = separation_course(test)

print(train.shape, test.shape)

(58344, 42) (6528, 42)


- 코스별 성적 스무딩

In [9]:
def apply_laplace_smoothing(df, col, global_mean, alpha):
    # 경기수 0인 값이 너무 높게 나오는 경향이 있어 분모에 상수 1 추가(없애도 됨)
    encoded_value = (df[f'{col}_성적'] * df[f'{col}_경기수'] + global_mean * alpha) / (1 + df[f'{col}_경기수'] + alpha)
    df[f'{col}_성적'] = encoded_value

    return df

def laplace_smoothing_to_course(train, val=None, alpha=1):
    col_list = [
        '1코스', '2코스', '3코스', '4코스', '5코스', '6코스'
    ]
    for col in col_list:
        train[f'{col}_성적'] = train[f'{col}_성적'].astype(float)
        train[f'{col}_경기수'] = train[f'{col}_경기수'].astype(float)
        if val is not None:
            val[f'{col}_성적'] = val[f'{col}_성적'].astype(float)
            val[f'{col}_경기수'] = val[f'{col}_경기수'].astype(float)

    # Train 데이터에서 글로벌 평균 계산
    global_means = {col: train[f'{col}_성적'].mean() for col in col_list}

    for col in col_list:
        # Train 데이터에 라플라스 스무딩 적용
        train = apply_laplace_smoothing(train, col, global_means[col], alpha)
        train.drop(f'{col}_경기수', axis=1, inplace=True)

    if val is not None:
        for col in col_list:
            # Validation 데이터에 Train에서 구한 글로벌 평균으로 라플라스 스무딩 적용
            val = apply_laplace_smoothing(val, col, global_means[col], alpha)
            val.drop(f'{col}_경기수', axis=1, inplace=True)

    if val is not None:
        return train, val
    else:
        return train


train, test = laplace_smoothing_to_course(train, test, alpha=1) # 알파가 작을수록 빈도수에 가깝세, 알파가 클수록 전체 평균에 가깝게
print(train.shape, test.shape)

(58344, 36) (6528, 36)


In [10]:
def set_course_scores(df):
    # '코스_성적' 열을 초기화
    df['코스_성적'] = 0

    # '번호' 열에 따른 '코스_성적' 값 설정
    for i in range(1, 7):
        mask = df['번호'] == i
        df.loc[mask, '코스_성적'] = df.loc[mask, f'{i}코스_성적']

    # 제거할 열 목록
    drop_cols = [f'{i}코스_성적' for i in range(1, 7)]

    # 열 제거
    df = df.drop(columns=drop_cols)

    return df


from sklearn.decomposition import PCA
def apply_pca(df_train, df_val, n_components=2):
    course_cols = [f'{i}코스_성적' for i in range(1, 7)]

    pca = PCA(n_components=n_components)
    pca.fit(df_train[course_cols])

    # 훈련 데이터에 PCA 변환 적용 (transform)
    train_pca = pca.transform(df_train[course_cols])
    val_pca = pca.transform(df_val[course_cols])
    for i in range(n_components):
        col_name = f'PCA_코스성적_{i+1}'
        df_train[col_name] = train_pca[:, i]
        df_val[col_name] = val_pca[:, i]

    # 각 컴포넌트별 분산 설명 비율 출력
    # explained_variance_ratios = pca.explained_variance_ratio_
    # for i, ratio in enumerate(explained_variance_ratios):
    #     print(f"PCA Component {i+1}: {ratio:.4f} variance explained")

    # n_components=2가 제일 좋아보임

    return df_train, df_val


train, test = apply_pca(train, test, n_components=2)
train = set_course_scores(train)
test = set_course_scores(test)

- 최근 8경기 착순 분리

In [11]:
def split_last_eight_rank(df):
    for i in range(0, 4):
        df[f'최근{i+1}경기_착순'] = df['최근8경주_착순'].str[i]
    for j in range(5, 9):
        df[f'최근{j}경기_착순'] = df['최근8경주_착순'].str[j]

    df.drop('최근8경주_착순', axis=1, inplace=True)

    return df

# def adjust_for_top3(df):
#     col_list = [
#         '최근1경기_착순', '최근2경기_착순', '최근3경기_착순', '최근4경기_착순',
#         '최근5경기_착순', '최근6경기_착순', '최근7경기_착순', '최근8경기_착순'
#     ]
# 
#     for col in col_list:
#     # 순위가 1, 2, 3이 아닌 경우, 결측인 경우, 6으로 조정
#     # (일반화된 성능을 위해 + 3등내에 드는게 중요)
#         df[col] = df[col].apply(lambda x: x if x in ['1', '2', '3'] else '-1')
# 
#     return df

def adjust_last_eight_rank(df_train, df_val):
    col_list = [
        '최근1경기_착순', '최근2경기_착순', '최근3경기_착순', '최근4경기_착순',
        '최근5경기_착순', '최근6경기_착순', '최근7경기_착순', '최근8경기_착순'
    ]

    for col in col_list:
        df_train[col] = df_train[col].fillna(6).astype(int)
        df_val[col] = df_val[col].fillna(6).astype(int)

        # df_train[col] = df_train[col].replace(0, 1)  # 가끔씩 0이 있는 경우가 존재
        # df_val[col] = df_val[col].replace(0, 1)
        # 그냥 냅두는게 성능이 젤 높음

        max_rank = df_train[col].max()
        df_train[col] = max_rank - df_train[col]
        df_val[col] = max_rank - df_val[col]

    first_mean_cols = 3
    col_to_mean_1 = col_list[:first_mean_cols]

    df_train[f'최근{first_mean_cols}경기_평균'] = df_train[col_to_mean_1].mean(axis=1)
    df_val[f'최근{first_mean_cols}경기_평균'] = df_val[col_to_mean_1].mean(axis=1)

    df_train.drop(col_list, axis=1, inplace=True)
    df_val.drop(col_list, axis=1, inplace=True)

    return df_train, df_val


train = split_last_eight_rank(train)
# train = adjust_for_top3(train)

test = split_last_eight_rank(test)
# test = adjust_for_top3(test)

train, test = adjust_last_eight_rank(train, test)

print(train.shape, test.shape)

(58344, 33) (6528, 33)


In [12]:
def encode_categorical_features(df_train, df_val, target_value='복승'):
    df_train['번호_등급'] = df_train['번호'].astype(str) + df_train['등급'].astype(str)
    df_val['번호_등급'] = df_val['번호'].astype(str) + df_val['등급'].astype(str)

    target_encoder = TargetEncoder(smoothing=2)
    target_encoder.fit(df_train['번호_등급'], df_train[target_value])

    df_train['등급'] = target_encoder.transform(df_train['번호_등급'])
    df_val['등급'] = target_encoder.transform(df_val['번호_등급'])

    df_train.drop('번호_등급', axis=1, inplace=True)
    df_val.drop('번호_등급', axis=1, inplace=True)

    train_racetype = df_train[['경기종류']].copy()
    val_racetype = df_val[['경기종류']].copy()

    train_racetype[['플라잉', '온라인']] = 0
    val_racetype[['플라잉', '온라인']] = 0

    train_racetype.loc[train_racetype['경기종류']=='플라잉', '플라잉'] = 1
    train_racetype.loc[train_racetype['경기종류']=='온라인', '온라인'] = 1

    val_racetype.loc[val_racetype['경기종류']=='플라잉', '플라잉'] = 1
    val_racetype.loc[val_racetype['경기종류']=='온라인', '온라인'] = 1

    train_racetype.drop('경기종류', axis=1, inplace=True)
    val_racetype.drop('경기종류', axis=1, inplace=True)

    df_train = pd.concat([train_racetype, df_train], axis=1)
    df_val = pd.concat([val_racetype, df_val], axis=1)

    df_train.drop('경기종류', axis=1, inplace=True)
    df_val.drop('경기종류', axis=1, inplace=True)

    return df_train, df_val

train, test = encode_categorical_features(train, test)
train

,플라잉,온라인,Race_ID,번호,등급,기수,선수명,성별,나이,최근6회차_평균착순점,...,보트_연대율,단승,복승,삼복승,중량부과여부,부과된중량,PCA_코스성적_1,PCA_코스성적_2,코스_성적,최근3경기_평균
0,1,0,2016_1_1_1,1,0.755632,1,정용진,0,45,3.33,...,0.0,0.0,0.0,1.0,0,0.0,16.574134,23.135649,72.466565,5.000000
1,1,0,2016_1_1_1,2,0.633299,13,김민준,0,25,3.83,...,0.0,1.0,1.0,1.0,0,0.0,36.239415,-14.821515,67.503244,4.666667
2,1,0,2016_1_1_1,3,0.219374,11,김현덕,0,35,6.69,...,0.0,0.0,0.0,0.0,0,0.0,-29.709277,-2.637123,15.483830,2.000000
3,1,0,2016_1_1_1,4,0.451251,8,한종석,0,35,3.14,...,0.0,0.0,1.0,1.0,0,0.0,39.980929,-28.473963,35.617032,4.666667
4,1,0,2016_1_1_1,5,0.181429,9,정재용,0,35,5.94,...,0.0,0.0,0.0,0.0,0,0.0,-17.803091,3.856657,3.421800,1.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58339,1,0,2023_52_2_17,2,0.373120,1,김국흠,0,55,5.17,...,29.4,0.0,0.0,1.0,0,0.0,-23.509572,24.573374,5.746564,1.333333
58340,1,0,2023_52_2_17,3,0.388318,7,배혜민,0,45,4.21,...,32.1,0.0,0.0,0.0,0,0.0,29.011377,5.689561,36.807518,4.000000
58341,1,0,2023_52_2_17,4,0.324412,12,김인혜,1,35,4.26,...,26.1,0.0,1.0,1.0,0,0.0,18.544591,27.481776,15.950548,2.333333
58342,1,0,2023_52_2_17,5,0.142238,15,엄광호,0,35,6.56,...,30.7,0.0,0.0,0.0,0,0.0,-42.750123,-3.619043,3.421800,0.666667


## 범주형 변수 확인


In [13]:
def cal_cat_cols(train, val=None):
    objective_cols = []

    # 특정 문자열이 포함된 열을 범주형 변수로 지정
    # cat_kerword_list = ['번호', '기수', '경기_착순']
    cat_kerword_list = ['번호', '경기종류', 'Race_ID']
    for col in train.columns:
        if any(sub in col for sub in cat_kerword_list):
            objective_cols.append(col)
            train[col] = train[col].astype('str')
            val[col] = val[col].astype('str')

    # 나머지 열에 대해 숫자형 변환 시도
    for col in train.columns:
        if col in objective_cols:
            continue  # 이미 범주형으로 처리된 열은 제외
        try:
            # 'float' 타입으로 변환 시도
            train[col] = train[col].astype('float')
            val[col] = val[col].astype('float')
        except:
            objective_cols.append(col)

    cat_features = list(set(objective_cols) - set(['rank', 'Race_ID']))
    cat_features = [feature for feature in cat_features if '단승' not in feature]
    cat_features = [feature for feature in cat_features if '복승' not in feature]
    cat_features = [feature for feature in cat_features if '삼복승' not in feature]

    return cat_features


cat_features = cal_cat_cols(train, test)
cat_features

['번호', '선수명']

## X, y 분리

In [14]:
drop_cols = [
    'Race_ID', '번호', '단승', '복승', '삼복승', '선수명', '기수', '플라잉', '온라인'
]

train_type = train[['Race_ID', '플라잉', '온라인']]  # 일차까지?
X_train = train.drop(drop_cols, axis=1)
y_train = train[['단승']]

test_type = test[['Race_ID', '플라잉', '온라인']]
X_test = test.drop(drop_cols, axis=1)
y_test = test[['단승']]

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(58344, 25) (58344, 1) (6528, 25) (6528, 1)


In [15]:
def reshape_race_data(df, players_per_race=6):
    # numpy 배열로 변환
    values = df.values

    # 데이터 형태 변환 (6개의 행을 1개의 행으로)
    reshaped_values = values.reshape(-1, players_per_race * values.shape[1])

    # 새로운 컬럼 이름 생성
    columns = [f'{col}_{i+1}번선수' for i in range(players_per_race) for col in df.columns]

    # 재구성된 DataFrame 생성
    reshaped_df = pd.DataFrame(reshaped_values, columns=columns)

    return reshaped_df

X_train = reshape_race_data(X_train, players_per_race=6)
y_train = reshape_race_data(y_train, players_per_race=6)
y_train = np.array(y_train).astype(int)

X_test = reshape_race_data(X_test, players_per_race=6)
y_test = reshape_race_data(y_test, players_per_race=6)
y_test = np.array(y_test).astype(int)

train_type.drop_duplicates(inplace=True)
test_type.drop_duplicates(inplace=True)

train_type = train_type.reset_index(drop=True)
test_type = test_type.reset_index(drop=True)

X_train = pd.concat([train_type, X_train], axis=1)
X_test = pd.concat([test_type, X_test], axis=1)

X_train.drop('Race_ID', axis=1, inplace=True)
X_test.drop('Race_ID', axis=1, inplace=True)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(9724, 152) (9724, 6) (1088, 152) (1088, 6)


In [16]:
def low_to_others(train, val, threshold=5, is_train=True, verbose=False):
    cat_features = cal_cat_cols(train, val)
    if is_train==False:
        cat_features = list(set(cat_features) - set(['Race_ID_1번선수', 'Race_ID_2번선수', 'Race_ID_3번선수', 'Race_ID_4번선수', 'Race_ID_5번선수', 'Race_ID_6번선수']))

    for col in cat_features:
        unifier = low_frequency_to_others(threshold=threshold, verbose=verbose)
        train[col] = unifier.fit_transform(train[col])
        val[col] = unifier.transform(val[col])

    return train, val

X_train, X_test = low_to_others(X_train, X_test, threshold=5, verbose=True)

In [17]:
def all_process(train, val, target='단승', is_train=True, is_dddd=False):
    train['성별'] = train['성별'].map({'남': 0, '여': 1})
    val['성별'] = val['성별'].map({'남': 0, '여': 1}) # 성별 mapping 남:0, 여:1

    train = add_y(train)
    if is_train:
        if is_dddd:
            val = add_y_(val)
        else:
            val = add_y(val)

    # train = add_weight_penalty(train) # 중량부과여부, 부과된 중량 추가 / 체중은 drop
    # val = add_weight_penalty(val) # 연승 정확도를 높이려면 제거

    # train = bin_age(train) # 나이 binning
    # val = bin_age(val)

    train = drop_columns_from_datasets(train)
    val = drop_columns_from_datasets(val)

    train, val = last_race_process(train, val) # 전일 성적 평균 추가

    train, val = reverse_rank_values(train, val) # 착순 관련 데이터 reverse

    train = separation_course(train)
    val = separation_course(val)

    train, val = laplace_smoothing_to_course(train, val, alpha=1)

    train, val = apply_pca(train, val, n_components=2) # 코스별 성적 pca + 해당 코스만 남김
    train = set_course_scores(train)
    val = set_course_scores(val)

    train = split_last_eight_rank(train)
    # train = adjust_for_top3(train)

    val = split_last_eight_rank(val)
    # val = adjust_for_top3(val)

    train, val = adjust_last_eight_rank(train, val) # 최근 8경기 중 3개는 평균, 나머지 버림

    train, val = encode_categorical_features(train, val)

    if is_train:
        drop_cols = [
            'Race_ID', '번호', '단승', '복승', '삼복승', '선수명', '기수', '플라잉', '온라인'
        ]
    else:
        drop_cols = [
            'Race_ID', '번호', '선수명', '기수', '플라잉', '온라인'
        ]

    train_type = train[['Race_ID', '플라잉', '온라인']]  # 일차까지? 
    val_type = val[['Race_ID', '플라잉', '온라인']]

    X_train = train.drop(drop_cols, axis=1)
    y_train = train[[target]]
    X_train = reshape_race_data(X_train, players_per_race=6)
    y_train = reshape_race_data(y_train, players_per_race=6)
    # y_train = np.array(y_train).astype(int)

    X_val = val.drop(drop_cols, axis=1)
    X_val = reshape_race_data(X_val, players_per_race=6)
    if is_train:
        y_val = val[[target]]
        y_val = reshape_race_data(y_val, players_per_race=6)
        # y_val = np.array(y_val).astype(int)

    train_type.drop_duplicates(inplace=True)
    val_type.drop_duplicates(inplace=True)

    train_type = train_type.reset_index(drop=True)
    val_type = val_type.reset_index(drop=True)

    X_train = pd.concat([train_type, X_train], axis=1)
    X_val = pd.concat([val_type, X_val], axis=1)

    X_train.drop('Race_ID', axis=1, inplace=True)
    X_val.drop('Race_ID', axis=1, inplace=True)

    X_train, X_val = low_to_others(X_train, X_val, threshold=5, is_train=is_train, verbose=False)

    if is_train:
        return X_train, y_train, X_val, y_val
    else:
        return X_train, y_train, X_val


def cal_y_val_for_연승(train, test):
    df_train = train.copy()
    df_test = test.copy()
    _, _, _, y_val = all_process(df_train, df_test, target='복승', is_train=True)

    return y_val


train = pd.read_csv(os.path.join(ROOT_DIR, "train.csv"))
test = pd.read_csv(os.path.join(ROOT_DIR, "test.csv"))

target_value = '복승'
seed = 42

X_train, y_train, X_test, y_test = all_process(train, test, target='복승', is_train=True)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(9724, 146) (9724, 6) (1088, 146) (1088, 6)


In [18]:
X_train.head()

,플라잉,온라인,등급_1번선수,성별_1번선수,나이_1번선수,체중_1번선수,최근6회차_평균착순점_1번선수,최근6회차_평균득점_1번선수,최근6회차_승률_1번선수,최근6회차_연대율2_1번선수,...,출주횟수_6번선수,모터_평균착순점_6번선수,모터_연대율2_6번선수,모터_연대율3_6번선수,보트_평균착순점_6번선수,보트_연대율_6번선수,PCA_코스성적_1_6번선수,PCA_코스성적_2_6번선수,코스_성적_6번선수,최근3경기_평균_6번선수
0,1.0,0.0,0.755632,0.0,44.0,56.0,3.33,3.40,40.0,40.0,...,33.0,10.0,0.0,0.0,10.0,0.0,-15.966908,-36.511023,12.682293,3.000000
1,1.0,0.0,0.755632,0.0,39.0,55.0,3.56,3.72,12.5,43.8,...,41.0,10.0,0.0,0.0,10.0,0.0,-11.700030,8.850585,1.560071,1.333333
2,1.0,0.0,0.616358,0.0,42.0,55.0,3.67,3.67,20.0,53.3,...,34.0,10.0,0.0,0.0,10.0,0.0,14.724391,-17.554835,30.577234,2.000000
3,1.0,0.0,0.526792,0.0,45.0,56.0,7.20,7.20,6.7,6.7,...,30.0,10.0,0.0,0.0,10.0,0.0,-28.204553,-8.589447,2.340106,1.333333
4,1.0,0.0,0.755632,0.0,38.0,57.0,2.11,2.17,50.0,66.7,...,23.0,10.0,0.0,0.0,10.0,0.0,-31.171677,12.920544,22.788127,1.666667


In [20]:
from catboost_function import evaluate_
from catboost_function import custom_CatBoostClassifier

def objective(trial):
    params_list = []
    params_1 = {
        'random_seed': seed,
        'iterations': 100,
        'learning_rate': trial.suggest_float('lr_1', 0.01, 0.15),
        'l2_leaf_reg': trial.suggest_float('l2_1', 0.1, 10),
        'scale_pos_weight': trial.suggest_float('pos_weight_1', 0.5, 1.5),
        'verbose': 0,
    }
    params_list.append(params_1)
    params_2 = {
        'random_seed': seed,
        'iterations': 100,
        'learning_rate': trial.suggest_float('lr_2', 0.01, 0.15),
        'l2_leaf_reg': trial.suggest_float('l2_2', 0.1, 10),
        'scale_pos_weight': trial.suggest_float('pos_weight_2', 0.5, 2.0),
        'verbose': 0,
    }
    params_list.append(params_2)
    params_3 = {
        'random_seed': seed,
        'iterations': 100,
        'learning_rate': trial.suggest_float('lr_3', 0.01, 0.15),
        'l2_leaf_reg': trial.suggest_float('l2_3', 0.1, 10),
        'scale_pos_weight': trial.suggest_float('pos_weight_3', 0.5, 2.0),
        'verbose': 0,
    }
    params_list.append(params_3)
    params_4 = {
        'random_seed': seed,
        'iterations': 100,
        'learning_rate': trial.suggest_float('lr_4', 0.01, 0.15),
        'l2_leaf_reg': trial.suggest_float('l2_4', 0.1, 10),
        'scale_pos_weight': trial.suggest_float('pos_weight_4', 0.8, 2.5),
        'verbose': 0,
    }
    params_list.append(params_4)
    params_5 = {
        'random_seed': seed,
        'iterations': 100,
        'learning_rate': trial.suggest_float('lr_5', 0.01, 0.15),
        'l2_leaf_reg': trial.suggest_float('l2_5', 0.1, 10),
        'scale_pos_weight': trial.suggest_float('pos_weight_5', 1.0, 3.0),
        'verbose': 0,
    }
    params_list.append(params_5)
    params_6 = {
        'random_seed': seed,
        'iterations': 100,
        'learning_rate': trial.suggest_float('lr_6', 0.01, 0.15),
        'l2_leaf_reg': trial.suggest_float('l2_6', 0.1, 10),
        'scale_pos_weight': trial.suggest_float('pos_weight_6', 1.0, 3.5),
        'verbose': 0,
    }
    params_list.append(params_6)

    smote_strategies = []
    smote_strategies.append(0.9)
    smote_strategies.append(0.8)
    smote_strategies.append(0.6)
    smote_strategies.append(0.5)
    smote_strategies.append(0.4)
    smote_strategies.append(0.3)

    train = pd.read_csv(os.path.join(ROOT_DIR, "train.csv"))
    test = pd.read_csv(os.path.join(ROOT_DIR, "test.csv"))
    unique_race_ids = train['Race_ID'].unique()
    kf = KFold(n_splits=5, shuffle=True, random_state=seed)

    scores = []
    연승_scores = []
    models = []
    for i, (train_race_ids, val_race_ids) in enumerate(kf.split(unique_race_ids)):
        train_ids = unique_race_ids[train_race_ids]
        val_ids = unique_race_ids[val_race_ids]

        train_fold = train[train['Race_ID'].isin(train_ids)].reset_index(drop=True)
        val_fold = train[train['Race_ID'].isin(val_ids)].reset_index(drop=True)
        test_fold = test.copy()

        val_fold = pd.concat([val_fold, test_fold], axis=0).reset_index(drop=True)

        y_val_for_연승 = cal_y_val_for_연승(train_fold, val_fold)

        X_train, y_train, X_val, y_val = all_process(train_fold, val_fold, target=target_value, is_train=True)

        model = custom_CatBoostClassifier(params_list, smote_strategies=smote_strategies)
        
        cat_features = cal_cat_cols(X_train, X_val)
        model.fit(
            X_train, y_train
        )

        models.append(model)
        y_pred = models[i].predict_proba(X_val)

        score, 연승_score = evaluate_(y_pred, y_val, target_value='복승', contain_연승=True, y_val_연승=y_val_for_연승)
        
        scores.append(score)
        연승_scores.append(연승_score)

    print('-'*50)
    print(f'AVG 연승 Score: {np.mean(연승_scores)}')
    print('-'*50)
    print('-'*50)

    return np.mean(연승_scores)

import optuna

study = optuna.create_study(direction='maximize')

study.optimize(objective, n_trials=500, show_progress_bar=True)
# study.optimize(objective, n_trials=500)

print('Best trial:')
trial = study.best_trial
print(f'  Value: {trial.value}')
print('  Params: ')
for key, value in trial.params.items():
    print(f'    {key}: {value}')

[I 2024-09-11 17:18:34,528] A new study created in memory with name: no-name-6876be0f-219a-4ed1-8a40-0055cbb4f5ca


  0%|          | 0/500 [00:00<?, ?it/s]

[W 2024-09-11 17:18:43,101] Trial 0 failed with parameters: {'lr_1': 0.1002638827051724, 'l2_1': 8.809857948084943, 'pos_weight_1': 1.3269744196266073, 'lr_2': 0.12601257617482786, 'l2_2': 6.2171323498427356, 'pos_weight_2': 1.4763111618161187, 'lr_3': 0.01074666796775285, 'l2_3': 5.0578485707128875, 'pos_weight_3': 1.556101041859693, 'lr_4': 0.05574676604636347, 'l2_4': 8.185882714635712, 'pos_weight_4': 1.5510196083581826, 'lr_5': 0.08681079569684326, 'l2_5': 9.347097332276139, 'pos_weight_5': 1.5045737371116759, 'lr_6': 0.12076309933548057, 'l2_6': 1.2599898829960243, 'pos_weight_6': 1.3136953760190708} because of the following error: KeyError(0).
Traceback (most recent call last):
  File "C:\Users\eunhak\anaconda3\envs\KBoat_optuna\lib\site-packages\pandas\core\indexes\base.py", line 3805, in get_loc
    return self._engine.get_loc(casted_key)
  File "index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "index.pyx", line 196, in pandas._libs.index.IndexEngine.get_

KeyError: 0